In [0]:
%sql
USE CATALOG basab_catalog_retail;
USE SCHEMA silver_tables;

In [0]:
%sql
select * from inventory_silver

In [0]:
%sql
select * from inventory_silver
where stock_level>100
order by stock_level desc

In [0]:
df_gold_pos_raw=spark.table('pos_silver')

In [0]:
df_gold_pos_raw.display()

In [0]:
from pyspark.sql.functions import *
from pyspark.sql.types import *

In [0]:
daily_sales=df_gold_pos_raw.groupBy('date').agg(
          sum("units_sold").alias("total_units_sold"),
          (sum(col("units_sold")*col("price"))).alias("total_revenue"),
          countDistinct("store_id").alias("active_stores"),
          sum(when(col("promo_flag")==1, col("units_sold")).otherwise(0)).alias("promo_units_sold")
      ).orderBy("date")

In [0]:
daily_sales = daily_sales.filter(
    (col("date").isNotNull()) & (col("date") != '1900-01-01')
)


In [0]:
daily_sales.limit(2).display()

In [0]:
daily_sales=daily_sales.withColumn("total_revenue",round(col("total_revenue"),2))

In [0]:
daily_sales = daily_sales.withColumn("day_name", date_format(col("date"), "EEEE"))

In [0]:
daily_sales.display()


In [0]:
day_wise_sale=daily_sales.groupBy("day_name").agg(round(sum("total_revenue"),2).alias("total_revenue")).orderBy(col("total_revenue").desc())

In [0]:
day_wise_sale.display()

In [0]:
%sql
create schema if not exists gold_tables

In [0]:
daily_sales.write \
    .format("delta") \
    .mode("overwrite") \
    .saveAsTable("basab_catalog_retail.gold_tables.daily_sales")

In [0]:
day_wise_sale.write \
    .format("delta") \
    .mode("overwrite") \
    .saveAsTable("basab_catalog_retail.gold_tables.day_wise_sale")

In [0]:
day_ordered = (
    daily_sales
    .withColumn(
        "day_order",
        when(col("day_name")=="Monday", 1)
        .when(col("day_name")=="Tuesday", 2)
        .when(col("day_name")=="Wednesday", 3)
        .when(col("day_name")=="Thursday", 4)
        .when(col("day_name")=="Friday", 5)
        .when(col("day_name")=="Saturday", 6)
        .when(col("day_name")=="Sunday", 7)
    )
)

import matplotlib.pyplot as plt

# Convert to Pandas
df_plot = day_ordered.select("day_name", "total_revenue", "day_order").toPandas()

# Sort by day_order
df_plot = df_plot.sort_values("day_order")

In [0]:
daily_sales.groupBy("day_name").agg(sum("promo_units_sold").alias("total_promo_units_sold")).orderBy(col("total_promo_units_sold").desc()).display()

In [0]:
plt.figure(figsize=(10,5))
plt.plot(df_plot["day_name"], df_plot["total_revenue"], marker='o', linestyle='-')
plt.title("Day-wise Total Revenue")
plt.xlabel("Day of Week")
plt.ylabel("Total Revenue")
plt.grid(True)
plt.show()

#### Day_wise_sales shows that Traffic is very high on Wednesday and Sunday , low on  Tuesdays ,Fridays and Saturdays 

In [0]:
holidays=spark.table('silver_tables.holiday_silver')


In [0]:
holidays.display()

In [0]:
holidays_renamed = holidays.withColumnRenamed("date", "holiday_date")

# Then join using the original daily_sales date
daily_sales_with_holiday = daily_sales.join(
    holidays_renamed,
    daily_sales.date == holidays_renamed.holiday_date,  # reference the renamed column
    "left"
)

daily_sales_with_holiday.filter(col("holiday_date").isNotNull()).display()

In [0]:
avg_total_revenue = daily_sales.select(avg("total_revenue").alias("avg_rev")).collect()[0]["avg_rev"]

# Filter the dataframe
daily_sales.filter(col("total_revenue") > avg_total_revenue).display()

In [0]:
stores=spark.table('silver_tables.stores_silver')


In [0]:
stores.display()

In [0]:
pos_stores_joined = df_gold_pos_raw.join(
    stores,
    df_gold_pos_raw.store_id == stores.store_id,
    "left"
).drop(stores.store_id)  # drop the 'store_id' from the right DataFrame

In [0]:
pos_stores_summary = pos_stores_joined.groupBy("store_id").agg(
    sum("units_sold").alias("total_units_sold"),
   round( sum(col("units_sold") * col("price")) ,2).alias("total_revenue")
).orderBy(col("total_units_sold").desc())

pos_stores_summary.display()

In [0]:
pos_stores_summary2=pos_stores_summary.join(stores,pos_stores_summary.store_id==stores.store_id,"inner").drop(stores.store_id)
pos_stores_summary2.display()

In [0]:
pos_stores_summary2.write.format('delta').mode('overwrite').saveAsTable('basab_catalog_retail.gold_tables.pos_stores_summary')

In [0]:
storeFormat_wise_unitSells=pos_stores_summary2.groupBy("format").agg(sum("total_units_sold").alias("total_units_sold")).orderBy(col("total_units_sold").desc())
storeFormat_wise_unitSells.display()



In [0]:
storeFormat_wise_unitSells.write.format('delta').mode('overwrite').saveAsTable('basab_catalog_retail.gold_tables.storeFormat_wise_unitSells')

In [0]:
region_wise_sales=pos_stores_summary2.groupBy('region').agg(sum('total_units_sold').alias('total_units_sold')).orderBy(col('total_units_sold').desc())

In [0]:
region_wise_sales.display()

In [0]:
region_wise_sales.write.format('delta').mode('overwrite').saveAsTable('basab_catalog_retail.gold_tables.region_wise_sales')